In [14]:
import pandas as pd
from datetime import datetime, timedelta

In [15]:
from selenium import webdriver 
from bs4 import BeautifulSoup

from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.chrome.service import Service as ChromeService
from urllib.error import URLError

In [16]:
import os

past_dates = []

for path in os.listdir('../Data/'):
    if path.split('.')[1] == "csv":
        past_dates += [
            '/20' + date.split('-')[0].split('.')[::-1][0] + 
                '/' +date.split('-')[0].split('.')[::-1][1] + 
                    '/' + date.split('-')[0].split('.')[::-1][2]
                        for date in pd.read_csv("../Data/" + path)['Day_month_year_hour']
                ]

print("total news : ", len(past_dates))
past_dates = list(set(past_dates))
print("total days :", len(past_dates))

total news :  162460
total days : 2222


In [17]:
max_day = 3000
start_day = 0
end_day = start_day + max_day

In [18]:
# Hesaplanan tarihlerin listesi
dates = []

def main(start_day, end_day, getToday):
    # Şu anki tarih ve saat bilgisini al
    current_date = datetime.now()
    
    if(getToday):
        # Şu anki yıl, ay ve gün bilgilerini al
        year = str(current_date.year)
        month = str(current_date.month)
        day = str(current_date.day)

        # Ay ve gün bilgilerini iki haneli yapma (01, 02, ..., 09)
        if len(month) == 1:
            month = "0" + month
        if len(day) == 1:
            day = "0" + day

        # Bugünün tarihini "/YYYY/MM/DD" formatında listeye ekleme
        dates.append("/" + year + "/" + month + "/" + day)
    
    # Belirtilen aralıkta tarihleri oluşturma
    for i in range(start_day, end_day):
        # Güncel tarihten önceki günleri hesapla
        previous_date = current_date - timedelta(days= i)

        # Önceki tarihin yıl, ay ve gün bilgilerini al
        year = str(previous_date.year)
        month = str(previous_date.month)
        day = str(previous_date.day)

        # Ay ve gün bilgilerini iki haneli yapma (01, 02, ..., 09)
        if len(month) == 1:
            month = "0" + month
        if len(day) == 1:
            day = "0" + day

        # Oluşturulan tarihi "/YYYY/MM/DD" formatında listeye ekleme
        date = "/" + year + "/" + month + "/" + day

        if date not in past_dates:
            dates.append(date)

    # Oluşturulan tarih listesini geri döndürme
    return dates

# Fonksiyonu çağırarak belirli bir gün aralığındaki tarihleri hesapla
dates = list(set(main(start_day=start_day, end_day=end_day, getToday=False)))

In [20]:
timeoutException_count = 0
webDriverException_count = 0
urlError_count = 0

# Belirli bir URL'den haber sayfasını çeken fonksiyon.
def get_news(url=""):
    global timeoutException_count
    global webDriverException_count 
    global urlError_count 
    
    # Web sürücüsü (driver) dosya yolunu belirleme
    driver_path = "../chromedriver-win32/chromedriver.exe"
    
    try:
        # Ana haber sitesi URL'si
        main_url = "https://www.sabah.com.tr"

        # Belirtilen URL'yi ana URL ile birleştirme
        full_url = main_url + url

        # Selenium servisini başlatma
        service = ChromeService(executable_path=driver_path)

        # WebDriver'ı başlatma
        driver = webdriver.Chrome(service=service)

        # Sayfa boyutunu ayarlama
        driver.set_window_size(10, 10)

        # Belirtilen URL'yi açma
        driver.get(full_url)

        # Sayfa kaynağını alıp BeautifulSoup kullanarak parse etme
        source = driver.page_source
        soup = BeautifulSoup(source, features="html.parser")

    except TimeoutException:
        timeoutException_count += 1
        return None

    except WebDriverException:
        webDriverException_count += 1
        return None

    except URLError:
        print(f"Bağlantı hatası: URLError")
        urlError_count += 1
        return None

    finally:
        # WebDriver'ı kapatma
        driver.quit()

    # Parse edilmiş HTML içeriğini geri döndürme
    return soup


In [21]:
# Haber içeriğini alan fonksiyon.
def get_content(content_soup, url=""):
    # Haber içeriğini tutacak boş bir string
    news_content = ""   
    # Belirli bir URL'ye özgü div yapısını içeren ana div'i bulma
    top_div = content_soup.find_all("div", attrs={"class": "col-sm-12 view20" + url})
    # Eğer belirli bir div yapısı bulunursa devam et
    if top_div is not None:
        # Her bir ana div içinde dolaş
        for div in top_div:
            # İç içe geçmiş div'ler arasında, içerik div'ini bulma
            inner_div = div.find("div", attrs={"class": "newsDetailText"})
            # Eğer içerik div'i bulunursa devam et
            if inner_div is not None:
                # İçerik div'inin içindeki detay div'ini bulma
                detail_div = inner_div.find("div", attrs={"class": "newsBox selectionShareable"})
                # Eğer detay div'i bulunursa devam et
                if detail_div is not None:
                    # Detay div'i içindeki paragrafları bulma
                    paragraphs = detail_div.find_all('p')
                    # Eğer paragraflar bulunursa devam et
                    if paragraphs is not None:
                        # Her bir paragrafı dolaşarak haber içeriğini birleştirme
                        for paragraph in paragraphs:
                            if paragraph is not None:
                                news_content += paragraph.text
    # Oluşturulan haber içeriğini geri döndürme
    return news_content

In [22]:
# 'roza' URL'sinden haber içeriğini alan fonksiyon.
def get_news_from_roza(content_url):
    # Haber içeriğini tutacak boş bir string
    news_content = ""

    try:
        # Belirli bir 'roza' URL'sine ait haber sayfasını çekme
        soup = get_news(url=content_url)

        # Haber içeriğini içeren div yapısını bulma
        paragraphs = soup.find("div", class_="detail-text-area")

        # Eğer belirli bir div yapısı bulunursa devam et
        if paragraphs is not None:
            # Her bir paragrafı dolaş
            for p in paragraphs.find_all("p"):
                # Eğer paragraf içinde "strong" etiketi yoksa, içeriği birleştir
                if not p.find("strong"):
                    news_content += p.get_text(strip=True) + " "

    except Exception as e:
        print(f"Error : {e}")

    # Oluşturulan haber içeriğini geri döndürme
    return news_content.strip()

In [23]:
def get_news_from_finans(content_url):
    news_content = ""
    
    soup = get_news(url=content_url)
    
    # Haber içeriğini içeren div yapısını bulma
    paragraphs = soup.find("div", attrs={"class" : "detail-text-area"})

    # Eğer belirli bir div yapısı bulunursa devam et
    if paragraphs is not None:
        # Her bir paragrafı dolaş
        for p in paragraphs.find_all("p"):
            # Eğer paragraf içinde "strong" etiketi yoksa, içeriği birleştir
            if p.find("strong") is None:
                news_content += p.text

    # Oluşturulan haber içeriğini geri döndürme
    return news_content

In [24]:
content_urls = []  
# Haber türlerini (örneğin, yaşam, ekonomi, spor) tutacak liste
news_types = []  
# Haber başlıklarını tutacak liste
titles = []  
# Haber tarih ve saat bilgilerini tutacak liste
date_time_list = []  
# Haber görsel URL'lerini tutacak liste
image_urls = []  
# Haber içeriklerini tutacak liste
contents = []  

datas = []
def append_data(title="", content='', content_url='', news_type='', date_time='', img_url=''):
    datas.append([title, content, content_url, news_type, date_time, img_url])
    return datas

In [25]:
# Geçerli URL'nin indeksi
current_url_index = 0  

# Yasaklı URL'ler listesi, farklı HTML yapılarına sahip olanlar
banned_urls = []
# Yasaklı haberlerin listesi
banned_news = []
# Haber içeriklerinin URL'lerini tutacak liste

# Yasaklı bağlantıların listesi
banned_hrefs = []

# Açilmayan sayfa sayısı
counter = 0

# Ana döngü, belirli bir tarih aralığındaki haberleri almak için kullanılır
while True:
    # Veriyi almak için URL'yi fonksiyona veriyoruz.
    soup = get_news(url="/timeline" + dates[current_url_index])
    
    # Sayfa açılmadıysa bu if çalışmaz ve current_url_index değişmez, böylece sayfa değişmez
    if soup is not None and len(soup.find_all('div', attrs={'class': 'box'})) != 0:
        
        # İlerlemeyi adım adım takip etmek için
        remaining_days = len(dates) - (current_url_index )
        if remaining_days % 10 == 0:
            print(f"{remaining_days} days left")  
        
        # news_boxes, o günün bütün haberlerini içeriyor
        for box in soup.find_all('div', attrs={'class': 'box'}):
            box_classes = box.get("class")

            # Her bir haberin class'i 'box', reklam olanların class'i 'box advert', uzunluğu 2 olduğu için reklamları almıyoruz
            if len(box_classes) == 2:
                pass

            # Eğer class sadece tek bir değerden oluşuyorsa ('box') o zaman bu haberi alıyoruz
            else:
                # Haber içeriğinin URL'sini al
                content_url = box.find('a').get("href")
                head_url = content_url.split("/")[1]        

                # Haberin türünü al
                news_type = box.find_all("div", attrs={"class": "info"})[0].find("a").text.strip()
                
                # Haber başlığını al
                title = box.find('strong').text.strip() 
                
                # Haber tarih ve saat bilgisini al
                current_date = box.find('div', {'class': "info"}).find("label").text.strip()
                date_time = current_date.split(" ")[0].split(",")[0] + '-' + current_date.split(" ")[-1]
                
                # Haber görsel URL'sini al
                image_url = box.find("img").get('src')

                # URL 'roza' ile başlıyorsa, özel bir fonksiyon kullanarak haber içeriğini al
                if head_url == "roza":

                    roza_news = get_news_from_roza(content_url)
                    data_arrays = append_data(title=title, content=roza_news, content_url=content_url, news_type=news_type, date_time=date_time, img_url=image_url)

                # 'roza' ile başlamayan ve yasaklı URL'ler içinde olmayan haber başlıklarını al
                elif head_url not in banned_urls:
                    content_soup = get_news(url=content_url)
                    if content_soup is not None:
                        
                        content_check = content_soup.find_all("div", attrs={"class": "col-sm-12 view20 pagedItems"})

                        # Eğer haber içeriğinde birden fazla fotoğraf varsa class yapısı değişiyor ve burada o haberin bu sayfa yapısına sahip olup 
                        # olmadığını kontrol ediyoruz 
                        if len(content_check) == 0:
                            news_content = get_content(content_soup=content_soup)
                            data_arrays = append_data(title=title, content=news_content, content_url=content_url, news_type=news_type, date_time=date_time, img_url=image_url)
                        # Eğer haber birden fazla fotoğraf ve içerikten oluşuyorsa bu şart sağlanır
                        else:    
                            news_content = get_content(content_soup=content_soup, url=" pagedItems")
                            data_arrays = append_data(title=title, content=news_content, content_url=content_url, news_type=news_type, date_time=date_time, img_url=image_url)
                
                elif head_url == "finans":
                    
                    finans_news = get_news_from_finans(content_url)
                    data_arrays = append_data(title=title, content=finans_news, content_url=content_url, news_type=news_type, date_time=date_time, img_url=image_url)
                    
                else:
                    # Eğer URL 'roza', 'finans' ve 'spor' sayfalarına giriyorsa geç
                    banned_hrefs.append(content_url)
                    
        current_url_index += 1        


    # Eğer sayfa açılmazsa current_url_index değişmez ve döngü tekrar aynı sayfayı deneyecektir
    else:
        counter += 1

    # İstenen gün sayısına ulaşıldığında döngüyü sonlandır
    if current_url_index == len(dates):
        break

200 days left


In [ ]:
print("timeoutException_count :" , timeoutException_count)
print("webDriverException_count :", webDriverException_count)
print('urlError_count :', urlError_count)

timeoutException_count : 0
webDriverException_count : 207
urlError_count : 0


In [ ]:
df = pd.DataFrame(columns=['Title', 'Content', 'Content_url', 'News_type', 'Day_month_year_hour', 'Img_url'])

for data in data_arrays: 
    df.loc[len(df)] = data

In [ ]:
df.to_csv("../Data/data_33.csv", encoding='utf-8', index=False)

In [ ]:
pd.read_csv('../Data/data_33.csv').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11302 entries, 0 to 11301
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Title                11302 non-null  object
 1   Content              7696 non-null   object
 2   Content_url          11302 non-null  object
 3   News_type            11302 non-null  object
 4   Day_month_year_hour  11302 non-null  object
 5   Img_url              11302 non-null  object
dtypes: object(6)
memory usage: 529.9+ KB
